# Import library

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import json
import argparse,random,logging,numpy,os
import torch.utils.data as data

import numpy as np
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm
from time import time
from tqdm import tqdm

from collections import OrderedDict
from glob import glob
from time import time
from multiprocessing import Pool,cpu_count
from itertools import chain

# Build model

## Build basic model

In [3]:
import torch
import os
from torch.autograd import Variable

class BasicModule(torch.nn.Module):
    def __init__(self, device, save_dir='/kaggle/working/checkpoints/', seed=1):
        super(BasicModule, self).__init__()  # Gọi hàm khởi tạo của lớp cha (torch.nn.Module)
        self.model_name = self.__class__.__name__  # Lưu tên của lớp hiện tại
        self.device = device  # Thiết bị (CPU/GPU) để chạy mô hình
        self.save_dir = save_dir  # Thư mục lưu trữ các checkpoint
        self.seed = seed if seed is not None else 0 

        # Kiểm tra và tạo thư mục nếu không tồn tại
        if not os.path.isdir(self.save_dir):
            os.makedirs(self.save_dir)

    def save(self):
        checkpoint = {'model': self.state_dict(), 'seed': self.seed}  # Tạo dictionary chứa trạng thái mô hình và seed
        best_path = f'{self.save_dir}/{self.model_name}_seed_{self.seed}.pt'  # Tạo đường dẫn lưu checkpoint
        torch.save(checkpoint, best_path)  # Lưu checkpoint
        return best_path  # Trả về đường dẫn lưu checkpoint

    def load(self, best_path):
        if self.device is not None:
            data = torch.load(best_path)['model']  # Tải checkpoint từ GPU/CPU
        else:
            data = torch.load(best_path, map_location=torch.device('cpu'))['model']  # Tải checkpoint từ CPU
        self.load_state_dict(data)  # Load trạng thái mô hình từ checkpoint
        return self.to(self.device)  # Chuyển mô hình sang thiết bị (CPU/GPU)
        
    def pad_doc(self, words_out, doc_lens):
        pad_dim = words_out.size(1)  # Chiều kích thước của từ ngữ
        max_doc_len = max(doc_lens)  # Độ dài tối đa của các đoạn văn bản
        sent_input = []  # Danh sách chứa các đoạn văn bản đã padding
        start = 0
        for doc_len in doc_lens:
            stop = start + doc_len
            valid = words_out[start:stop]  # Lấy ra đoạn văn bản có độ dài tương ứng
            start = stop
            if doc_len == max_doc_len:
                sent_input.append(valid.unsqueeze(0))  # Thêm đoạn văn bản vào danh sách nếu không cần padding
            else:
                pad = torch.zeros(max_doc_len - doc_len, pad_dim).to(self.device)  # Tạo padding cho đoạn văn bản
                sent_input.append(torch.cat([valid, pad]).unsqueeze(0))  # Thêm đoạn văn bản đã được padding vào danh sách
        sent_input = torch.cat(sent_input, dim=0)  # Gộp tất cả các đoạn văn bản đã padding thành một tensor
        return sent_input


## Build CNN-RNN

In [4]:

class CNN_RNN(BasicModule):
    def __init__(self, device, kernel_sizes, embed_dim, kernel_num, embed_num, hidden_size, seg_num, pos_num, pos_dim, embed=None):
        super(CNN_RNN, self).__init__(device)  # Gọi hàm khởi tạo của lớp cha (BasicModule)

        self.model_name = 'CNN_RNN'  # Đặt tên cho mô hình
        self.hidden_size = hidden_size  # Kích thước lớp ẩn của RNN
        self.device = device  # Thiết bị (CPU/GPU) để chạy mô hình

        # Cài đặt thông số cho các biến
        Ks = kernel_sizes  # Kích thước kernel của CNN
        Ci = embed_dim  # Kích thước embedding của từ
        Co = kernel_num  # Số lượng kernel của CNN
        V = embed_num  # Số lượng từ trong từ điển
        D = embed_dim  # Kích thước embedding của từ
        H = hidden_size  # Kích thước lớp ẩn của RNN
        S = seg_num  # Số lượng phân đoạn của văn bản
        P_V = pos_num  # Số lượng vị trí tuyệt đối
        P_D = pos_dim  # Kích thước embedding của vị trí

        self.abs_pos_embed = nn.Embedding(P_V, P_D)  # Embedding cho vị trí tuyệt đối
        self.rel_pos_embed = nn.Embedding(S, P_D)  # Embedding cho vị trí tương đối
        self.embed = nn.Embedding(V, D, padding_idx=0)  # Embedding cho từ, với padding index
        if embed is not None:
            self.embed.weight.data.copy_(embed)  # Nếu có embedding được cung cấp, sao chép vào embedding layer

        # Danh sách các lớp CNN
        self.convs = nn.ModuleList([nn.Sequential(
            nn.Conv1d(Ci, Co, K),  # Lớp Convolutional 1D
            nn.BatchNorm1d(Co),  # Batch normalization
            nn.LeakyReLU(inplace=True),  # Hàm kích hoạt Leaky ReLU

            nn.Conv1d(Co, Co, K),  # Lớp Convolutional 1D thứ hai
            nn.BatchNorm1d(Co),  # Batch normalization
            nn.LeakyReLU(inplace=True)  # Hàm kích hoạt Leaky ReLU
        ) for K in Ks])

        # Lớp RNN (GRU)
        self.sent_RNN = nn.GRU(
            input_size=Co * len(Ks),
            hidden_size=H,
            batch_first=True,
            bidirectional=True
        )
        
        # Lớp Fully Connected
        self.fc = nn.Sequential(
            nn.Linear(2 * H, 2 * H),
            nn.BatchNorm1d(2 * H),
            nn.Tanh()
        )

        # Các lớp phân loại
        self.content = nn.Linear(2 * H, 1, bias=False)
        self.salience = nn.Bilinear(2 * H, 2 * H, 1, bias=False)
        self.novelty = nn.Bilinear(2 * H, 2 * H, 1, bias=False)
        self.abs_pos = nn.Linear(P_D, 1, bias=False)
        self.rel_pos = nn.Linear(P_D, 1, bias=False)
        self.bias = nn.Parameter(torch.FloatTensor(1).uniform_(-0.1, 0.1))  # Khởi tạo bias
    
    def max_pool1d(self, x, seq_lens):
        # x:[N, L, O_in]  (N: batch size, L: độ dài chuỗi tối đa, O_in: số chiều kênh đầu vào)
        out = []
        for index, t in enumerate(x):
            # Lấy phần dữ liệu hợp lệ của chuỗi (loại bỏ padding)
            t = t[:seq_lens[index], :]
            # Chuyển vị và thêm chiều để phù hợp với F.max_pool1d
            t = torch.t(t).unsqueeze(0)
            # Thực hiện max pooling 1D và thêm vào danh sách kết quả
            out.append(F.max_pool1d(t, t.size(2)))
    
        # Ghép các kết quả và loại bỏ chiều không cần thiết
        out = torch.cat(out).squeeze(2)
        return out
    
    def avg_pool1d(self, x, seq_lens):
        # Tương tự như max_pool1d, nhưng sử dụng average pooling
        out = []
        for index, t in enumerate(x):
            t = t[:seq_lens[index], :]
            t = torch.t(t).unsqueeze(0)
            out.append(F.avg_pool1d(t, t.size(2)))
    
        out = torch.cat(out).squeeze(2)
        return out
    
    def forward(self, x, doc_lens):
        # x:[N, L, D] (N: batch size, L: độ dài câu tối đa, D: chiều embedding)
        # doc_lens: danh sách độ dài tài liệu (số lượng câu trong mỗi tài liệu)
    
        # Tính toán độ dài câu (số lượng token khác 0)
        sent_lens = torch.sum(torch.sign(x), dim=1).data
        H = self.hidden_size
    
        # Embedding các token
        x = self.embed(x)  # (N, L, D)
    
        # Word-level CNN: áp dụng các lớp CNN và max pooling
        x = [conv(x.permute(0, 2, 1)) for conv in self.convs]
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        x = torch.cat(x, 1)
    
        # Padding tài liệu để có độ dài bằng nhau
        x = self.pad_doc(x, doc_lens)
    
        # Sentence-level GRU: áp dụng GRU ở cấp câu
        sent_out = self.sent_RNN(x)[0]  # (B, max_doc_len, 2*H)
    
        # Document-level Max Pooling: tạo vector biểu diễn tài liệu
        docs = self.max_pool1d(sent_out, doc_lens)  # (B, 2*H)
    
        # Fully Connected Layer: biến đổi vector biểu diễn tài liệu
        docs = self.fc(docs)
    
        probs = []
        for index, doc_len in enumerate(doc_lens):
            # Lấy các vector biểu diễn câu hợp lệ
            valid_hidden = sent_out[index, :doc_len, :]  # (doc_len, 2*H)
            doc = docs[index].unsqueeze(0)
            s = Variable(torch.zeros(1, 2*H))
            if self.device is not None:
                s = s.cuda()
    
            for position, h in enumerate(valid_hidden):
                h = h.view(1, -1)  # (1, 2*H)
    
                # Tính toán positional embeddings (absolute và relative)
                abs_index = Variable(torch.LongTensor([[position]]))
                if self.device is not None:
                    abs_index = abs_index.cuda()
                abs_features = self.abs_pos_embed(abs_index).squeeze(0)
    
                rel_index = int(round((position + 1) * 9.0 / doc_len))
                rel_index = Variable(torch.LongTensor([[rel_index]]))
                if self.device is not None:
                    rel_index = rel_index.cuda()
                rel_features = self.rel_pos_embed(rel_index).squeeze(0)
    
                # Tính toán điểm số cho câu
                content = self.content(h)
                salience = self.salience(h, doc)
                novelty = -1 * self.novelty(h, F.tanh(s))
                abs_p = self.abs_pos(abs_features)
                rel_p = self.rel_pos(rel_features)
    
                # Tính toán xác suất chọn câu
                prob = F.sigmoid(content + salience + novelty + abs_p + rel_p + self.bias)
                s = s + torch.mm(prob, h)
                probs.append(prob)
    
        # Ghép các xác suất và trả về kết quả
        return torch.cat(probs).squeeze()

## Build RNN-RNN

In [5]:
class RNN_RNN(BasicModule):
    def __init__(self, device, embed_num, embed_dim, hidden_size, seg_num, pos_num, pos_dim):
        super(RNN_RNN, self).__init__(device)

        self.model_name = 'RNN_RNN'
        self.device = device
        self.hidden_size = hidden_size

        # Cài đặt thông số cho các biến khác
        V = embed_num  # Số lượng từ vựng
        D = embed_dim  # Kích thước embedding của từ
        H = hidden_size # Kích thước ẩn của RNN
        S = seg_num   # Số lượng segment (vị trí tương đối)
        P_V = pos_num # Số lượng vị trí tuyệt đối
        P_D = pos_dim # Kích thước embedding vị trí

        # Embedding cho vị trí tuyệt đối và tương đối
        self.abs_pos_embed = nn.Embedding(P_V, P_D)
        self.rel_pos_embed = nn.Embedding(S, P_D)

        # Embedding cho từ
        self.embed = nn.Embedding(V, D, padding_idx=0)
        # Nếu có embedding được khởi tạo trước, copy vào lớp embedding
        if embed is not None:
            self.embed.weight.data.copy_(embed)

        # RNN cấp từ (word-level RNN)
        self.word_RNN = nn.GRU(
            input_size=D,
            hidden_size=H,
            batch_first=True,
            bidirectional=True
        )

        # RNN cấp câu (sentence-level RNN)
        self.sent_RNN = nn.GRU(
            input_size=2*H,
            hidden_size=H,
            batch_first=True,
            bidirectional=True
        )

        # Lớp fully connected để biến đổi vector biểu diễn tài liệu
        self.fc = nn.Linear(2*H, 2*H)

        # Parameters of Classification Layer (tham số của lớp phân loại)
        self.content = nn.Linear(2*H, 1, bias=False) # Tính điểm nội dung
        self.salience = nn.Bilinear(2*H, 2*H, 1, bias=False) # Tính điểm độ quan trọng
        self.novelty = nn.Bilinear(2*H, 2*H, 1, bias=False) # Tính điểm độ mới
        self.abs_pos = nn.Linear(P_D, 1, bias=False) # Tính điểm vị trí tuyệt đối
        self.rel_pos = nn.Linear(P_D, 1, bias=False) # Tính điểm vị trí tương đối
        self.bias = nn.Parameter(torch.FloatTensor(1).uniform_(-0.1, 0.1)) # Bias cho lớp phân loại

    def max_pool1d(self, x, seq_lens):
        # x:[N, L, O_in] (N: batch size, L: độ dài chuỗi tối đa, O_in: số chiều kênh đầu vào)
        out = []
        for index, t in enumerate(x):
            # Lấy phần dữ liệu hợp lệ của chuỗi (loại bỏ padding)
            t = t[:seq_lens[index], :]
            # Chuyển vị và thêm chiều để phù hợp với F.max_pool1d
            t = torch.t(t).unsqueeze(0)
            # Thực hiện max pooling 1D và thêm vào danh sách kết quả
            out.append(F.max_pool1d(t, t.size(2)))

        # Ghép các kết quả và loại bỏ chiều không cần thiết
        out = torch.cat(out).squeeze(2)
        return out

    def avg_pool1d(self, x, seq_lens):
        # Tương tự như max_pool1d, nhưng sử dụng average pooling
        out = []
        for index, t in enumerate(x):
            t = t[:seq_lens[index], :]
            t = torch.t(t).unsqueeze(0)
            out.append(F.avg_pool1d(t, t.size(2)))

        out = torch.cat(out).squeeze(2)
        return out

    def forward(self, x, doc_lens):
        # x:[N, L, D] (N: batch size, L: độ dài câu tối đa, D: chiều embedding)
        # doc_lens: danh sách độ dài tài liệu (số lượng câu trong mỗi tài liệu)

        # Tính toán độ dài câu (số lượng token khác 0)
        sent_lens = torch.sum(torch.sign(x), dim=1).data

        # Embedding các token
        x = self.embed(x)  # (N, L, D)

        # Word-level GRU: áp dụng GRU ở cấp từ
        H = self.hidden_size
        x = self.word_RNN(x)[0]  # (N, L, 2*H)

        # Max pooling trên kết quả của word-level GRU
        word_out = self.max_pool1d(x, sent_lens)

        # Padding tài liệu để có độ dài bằng nhau
        x = self.pad_doc(word_out, doc_lens)

        # Sentence-level GRU: áp dụng GRU ở cấp câu
        sent_out = self.sent_RNN(x)[0]  # (B, max_doc_len, 2*H)

        # Max pooling trên kết quả của sentence-level GRU
        docs = self.max_pool1d(sent_out, doc_lens)  # (B, 2*H)

        probs = []
        for index, doc_len in enumerate(doc_lens):
            # Lấy các vector biểu diễn câu hợp lệ
            valid_hidden = sent_out[index, :doc_len, :]  # (doc_len, 2*H)
            # Áp dụng lớp fully connected và hàm tanh cho vector biểu diễn tài liệu
            doc = F.tanh(self.fc(docs[index])).unsqueeze(0)
            s = Variable(torch.zeros(1, 2*H))
            if self.device is not None:
                s = s.cuda()

            for position, h in enumerate(valid_hidden):
                h = h.view(1, -1)  # (1, 2*H)

                # Tính toán positional embeddings (absolute và relative)
                abs_index = Variable(torch.LongTensor([[position]]))
                if self.device is not None:
                    abs_index = abs_index.cuda()
                abs_features = self.abs_pos_embed(abs_index).squeeze(0)

                rel_index = int(round((position + 1) * 9.0 / doc_len))
                rel_index = Variable(torch.LongTensor([[rel_index]]))
                if self.device is not None:
                    rel_index = rel_index.cuda()
                rel_features = self.rel_pos_embed(rel_index).squeeze(0)

                # Tính toán điểm số cho câu (classification layer)
                content = self.content(h)
                salience = self.salience(h, doc)
                novelty = -1 * self.novelty(h, F.tanh(s))
                abs_p = self.abs_pos(abs_features)
                rel_p = self.rel_pos(rel_features)

                # Tính toán xác suất chọn câu
                prob = F.sigmoid(content + salience + novelty + abs_p + rel_p + self.bias)
                s = s + torch.mm(prob, h)
                probs.append(prob)

        # Ghép các xác suất và trả về kết quả
        return torch.cat(probs).squeeze()

# Setup variabel

In [6]:
import logging

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s [INFO] %(message)s')
root_dir = "/kaggle/working"

# Thiết lập các biến
save_dir = f'{root_dir}/checkpoints/'
embed_dim = 100
embed_num = 100
pos_dim = 50
pos_num = 100
seg_num = 10
kernel_num = 100
kernel_sizes = [3,4,5]
model = 'RNN_RNN'
hidden_size = 200
vocab_dir = f'{root_dir}/data/vocab.npz'

# Train
lr = 1e-3
batch_size = 32
epochs = 6
seed = 1
train_dir = f'{root_dir}/data/train.json'
val_dir = f'{root_dir}/data/val.json'
embedding_dir = f'{root_dir}/data/embedding.npz'
word2id_dir = f'{root_dir}/data/word2id.json'
report_every = 1500
seq_trunc = 50
max_norm = 1.0

# Test
load_dir = f'{root_dir}/checkpoints/RNN_RNN_seed_1.pt'
test_dir = f'{root_dir}/data/test.json'
ref_dir = f'{root_dir}/outputs/ref'
hyp_dir = f'{root_dir}/outputs/hyp'
filename_dir = f'{root_dir}/x.txt'  # TextFile to be summarized
topk = 3

# Device
device = 0  # Giả sử bạn sử dụng GPU với id 0, hoặc đặt None nếu dùng CPU

# Option flags
test = True  # Set True nếu bạn muốn kích hoạt chế độ test
debug = False  # Set True nếu bạn muốn bật chế độ debug
predict = False  # Set True nếu bạn muốn bật chế độ dự đoán

# Kiểm tra xem có sử dụng GPU không
use_gpu = device is not None

# In các biến ra để kiểm tra
print(f"Save Directory: {save_dir}")
print(f"Embedding Dimension: {embed_dim}")
print(f"Use GPU: {use_gpu}")
print(f"Test Mode: {test}")
print(f"Debug Mode: {debug}")



# Kiểm tra nếu có GPU và nếu không sử dụng GPU thì cảnh báo
if torch.cuda.is_available() and not use_gpu:
    print("WARNING: You have a CUDA device, should run with -device 0")

# Đặt GPU và seed
if use_gpu:
    torch.cuda.set_device(device)  # Sử dụng giá trị 'device' đã đặt trước đó
    torch.cuda.manual_seed(seed)   # Sử dụng giá trị 'seed' đã đặt trước đó

torch.manual_seed(seed)
random.seed(seed)
numpy.random.seed(seed)

# In ra thông tin nếu cần để kiểm tra
print(f"Using GPU: {use_gpu}, Device ID: {device if use_gpu else 'None'}")
print(f"Random Seed: {seed}")

Save Directory: /kaggle/working/checkpoints/
Embedding Dimension: 100
Use GPU: True
Test Mode: True
Debug Mode: False
Using GPU: True, Device ID: 0
Random Seed: 1


# Preprocessing Data

In [7]:
!gdown --id 1JgsboIAs__r6XfCbkDWgmberXJw8FBWE

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1JgsboIAs__r6XfCbkDWgmberXJw8FBWE
From (redirected): https://drive.google.com/uc?id=1JgsboIAs__r6XfCbkDWgmberXJw8FBWE&confirm=t&uuid=8d6d8bb0-2b68-4ba3-b100-a8dd1e1945b8
To: /kaggle/working/data.tar.gz
100%|████████████████████████████████████████| 375M/375M [00:04<00:00, 76.7MB/s]


In [8]:
!tar -xvzf /kaggle/working/data.tar.gz -C /kaggle/working/

data/
data/embedding.npz
data/test.json
data/train.json
data/val.json
data/word2id.json


In [9]:
with open(train_dir) as f: # Tải dữ liệu train
    train_examples = [json.loads(line) for line in f]
len(train_examples)

193983

In [10]:
train_examples[2]

{'doc': "by thomas durante a newly - released surveillance video taken from inside an oklahoma prison shows a former crime boss and key federal witness lured to his death by fellow inmates\nsix inmates have been charged with murder in what looks like a pre-planned attack in which they worked together , and even celebrated afterwards\nmollman , a former motorcycle gang member and admitted drug trafficker , was found in a cell at the grady county on may 17 , 2012\nhe had pleaded guilty to various drug charges and was a key witness for the government in an ongoing drug investigation\nthe six inmates charged in mollman 's murder have been identified as sitlington , grant curry , matthew jackson , joseph hill , jerry gonzales and steven nidey\nsitlington , whom grady county assistant district attorney leah edwards calls one of the primary attackers , is the first inmate to appear before a judge\nhis next court date is next month\nshe told news 9 : ' although much of what went on is obscured

In [11]:
def build_vocab():
    """
    Xây dựng từ vựng (vocabulary) từ file embedding được cung cấp.
    """
    print('start building vocab')

    # Định nghĩa các token đặc biệt
    PAD_IDX = 0  # Index cho token padding
    UNK_IDX = 1  # Index cho token unknown
    PAD_TOKEN = 'PAD_TOKEN' # Token padding
    UNK_TOKEN = 'UNK_TOKEN' # Token unknown

    # Mở file embedding để đọc
    f = open(embedding_dir)
    # Đọc dòng đầu tiên để lấy kích thước embedding
    embed_dim = int(next(f).split()[1])

    # Khởi tạo OrderedDict để lưu từ và index của chúng
    word2id = OrderedDict()

    # Thêm các token đặc biệt vào từ vựng
    word2id[PAD_TOKEN] = PAD_IDX
    word2id[UNK_TOKEN] = UNK_IDX

    # Khởi tạo list để lưu các vector embedding
    embed_list = []
    # Thêm vector 0 cho PAD và UNK
    embed_list.append([0 for _ in range(embed_dim)])
    embed_list.append([0 for _ in range(embed_dim)])

    # Xây dựng từ vựng từ file embedding
    for line in f:
        tokens = line.split()
        # Lấy từ và vector embedding
        word = tokens[:-1*embed_dim][0]
        vector = [float(num) for num in tokens[-1*embed_dim:]]
        # Thêm vector embedding vào list
        embed_list.append(vector)
        # Thêm từ vào từ vựng với index tương ứng
        word2id[word] = len(word2id)
    # Đóng file embedding
    f.close()
    # Chuyển list embedding thành numpy array
    embed = np.array(embed_list, dtype=np.float32)
    # Lưu numpy array embedding vào file npy
    np.savez_compressed(file=vocab_dir, embedding=embed)
    # Lưu từ vựng (word2id) vào file json
    with open(word2id_dir, 'w') as f:
        json.dump(word2id, f)

def worker(files):
    """
    Xử lý một nhóm các file dữ liệu để tạo ra các ví dụ (examples).
    """
    examples = []
    for f in files:
        # Đọc nội dung file và chia thành các phần
        parts = open(f, encoding='latin-1').read().split('\n\n')
        try:
            # Lấy các entities từ phần cuối của file
            entities = {line.strip().split(':')[0]: line.strip().split(':')[1].lower() for line in parts[-1].split('\n')}
        except:
            # Nếu có lỗi khi xử lý entities, bỏ qua file này
            continue
        sents, labels, summaries = [], [], []
        # Xử lý phần nội dung (content)
        for line in parts[1].strip().split('\n'):
            content, label = line.split('\t\t\t')
            tokens = content.strip().split()
            # Thay thế các entities trong tokens bằng giá trị tương ứng
            for i, token in enumerate(tokens):
                if token in entities:
                    tokens[i] = entities[token]
            # Chuyển label thành '1' hoặc '0'
            label = '1' if label == '1' else '0'
            # Thêm câu và label vào list
            sents.append(' '.join(tokens))
            labels.append(label)
        # Xử lý phần summary
        for line in parts[2].strip().split('\n'):
            tokens = line.strip().split()
            # Thay thế các entities trong tokens bằng giá trị tương ứng
            for i, token in enumerate(tokens):
                if token in entities:
                    tokens[i] = entities[token]
            # Loại bỏ ký tự '*' và thêm summary vào list
            line = ' '.join(tokens).replace('*', '')
            summaries.append(line)
        # Tạo example dictionary và thêm vào list examples
        ex = {'doc': '\n'.join(sents), 'labels': '\n'.join(labels), 'summaries': '\n'.join(summaries)}
        examples.append(ex)
    return examples

def build_dataset(args):
    """
    Xây dựng dataset từ các file dữ liệu sử dụng multiprocessing.
    """
    t1 = time()

    print('start building dataset')
    # Cảnh báo nếu worker_num là 1 và có nhiều CPU
    if worker_num == 1 and cpu_count() > 1:
        print('[INFO] There are %d CPUs in your device, please increase -worker_num to speed up' % (cpu_count()))
        print("    It's a IO intensive application, so 2~10 may be a good choise")

    # Lấy danh sách các file dữ liệu
    files = glob(args.source_dir)
    data_num = len(files)
    # Chia các file thành các nhóm cho từng worker
    group_size = data_num // worker_num
    groups = []
    for i in range(worker_num):
        if i == worker_num - 1:
            groups.append(files[i*group_size:])
        else:
            groups.append(files[i*group_size: (i+1)*group_size])
    # Tạo pool các worker
    p = Pool(processes=worker_num)
    # Gửi các nhóm file cho các worker để xử lý
    multi_res = [p.apply_async(worker, (fs,)) for fs in groups]
    # Lấy kết quả từ các worker
    res = [res.get() for res in multi_res]

    # Ghi kết quả vào file json
    with open(target_dir, 'w') as f:
        for row in chain(*res):
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

    t2 = time()
    print('Time Cost : %.1f seconds' % (t2 - t1))

## Get feature from word2id

In [12]:
class Vocab():
    def __init__(self, embed, word2id):
        """
        Khởi tạo đối tượng Vocab.

        Args:
            embed (numpy.ndarray): Ma trận embedding từ vựng.
            word2id (dict): Từ điển ánh xạ từ từ sang index.
        """
        self.embed = embed
        self.word2id = word2id
        self.id2word = {v: k for k, v in word2id.items()}  # Tạo từ điển ánh xạ từ index sang từ
        assert len(self.word2id) == len(self.id2word)  # Đảm bảo kích thước từ điển nhất quán
        self.PAD_IDX = 0  # Index cho token padding
        self.UNK_IDX = 1  # Index cho token unknown
        self.PAD_TOKEN = 'PAD_TOKEN'  # Token padding
        self.UNK_TOKEN = 'UNK_TOKEN'  # Token unknown

    def __len__(self):
        """
        Trả về kích thước từ vựng.
        """
        return len(self.word2id)

    def i2w(self, idx):
        """
        Chuyển đổi index thành từ.

        Args:
            idx (int): Index của từ.

        Returns:
            str: Từ tương ứng với index.
        """
        return self.id2word[idx]

    def w2i(self, w):
        """
        Chuyển đổi từ thành index.

        Args:
            w (str): Từ cần chuyển đổi.

        Returns:
            int: Index của từ, hoặc UNK_IDX nếu từ không có trong từ vựng.
        """
        if w in self.word2id:
            return self.word2id[w]
        else:
            return self.UNK_IDX

    def make_features(self, batch, sent_trunc=50, doc_trunc=100, split_token='\n'):
        """
        Tạo features từ batch dữ liệu cho quá trình huấn luyện.

        Args:
            batch (dict): Batch dữ liệu chứa các trường 'doc', 'labels', 'summaries'.
            sent_trunc (int): Độ dài tối đa của câu.
            doc_trunc (int): Số lượng câu tối đa trong một tài liệu.
            split_token (str): Token dùng để tách câu trong tài liệu và labels.

        Returns:
            tuple: Tuple chứa features, targets, summaries, và doc_lens.
                - features (torch.LongTensor): Tensor chứa index của các từ trong câu.
                - targets (torch.LongTensor): Tensor chứa labels của các câu.
                - summaries (list): List chứa tóm tắt của các tài liệu.
                - doc_lens (list): List chứa độ dài của các tài liệu (số lượng câu).
        """
        sents_list, targets, doc_lens = [], [], []
        # Truncate document
        for doc, label in zip(batch['doc'], batch['labels']):
            sents = doc.split(split_token)
            labels = label.split(split_token)
            labels = [int(l) for l in labels]
            max_sent_num = min(doc_trunc, len(sents))
            sents = sents[:max_sent_num]
            labels = labels[:max_sent_num]
            sents_list += sents
            targets += labels
            doc_lens.append(len(sents))
        # Truncate or pad sentence
        max_sent_len = 0
        batch_sents = []
        for sent in sents_list:
            words = sent.split()
            if len(words) > sent_trunc:
                words = words[:sent_trunc]
            max_sent_len = len(words) if len(words) > max_sent_len else max_sent_len
            batch_sents.append(words)

        features = []
        for sent in batch_sents:
            feature = [self.w2i(w) for w in sent] + [self.PAD_IDX for _ in range(max_sent_len - len(sent))]
            features.append(feature)

        features = torch.LongTensor(features)
        targets = torch.LongTensor(targets)
        summaries = batch['summaries']

        return features, targets, summaries, doc_lens

    def make_predict_features(self, batch, sent_trunc=150, doc_trunc=100, split_token='. '):
        """
        Tạo features từ batch dữ liệu cho quá trình dự đoán.

        Args:
            batch (list): List chứa các tài liệu (mỗi tài liệu là một chuỗi).
            sent_trunc (int): Độ dài tối đa của câu.
            doc_trunc (int): Số lượng câu tối đa trong một tài liệu.
            split_token (str): Token dùng để tách câu trong tài liệu.

        Returns:
            tuple: Tuple chứa features và doc_lens.
                - features (torch.LongTensor): Tensor chứa index của các từ trong câu.
                - doc_lens (list): List chứa độ dài của các tài liệu (số lượng câu).
        """
        sents_list, doc_lens = [], []
        for doc in batch:
            sents = doc.split(split_token)
            max_sent_num = min(doc_trunc, len(sents))
            sents = sents[:max_sent_num]
            sents_list += sents
            doc_lens.append(len(sents))
        # Truncate or pad sentence
        max_sent_len = 0
        batch_sents = []
        for sent in sents_list:
            words = sent.split()
            if len(words) > sent_trunc:
                words = words[:sent_trunc]
            max_sent_len = len(words) if len(words) > max_sent_len else max_sent_len
            batch_sents.append(words)

        features = []
        for sent in batch_sents:
            feature = [self.w2i(w) for w in sent] + [self.PAD_IDX for _ in range(max_sent_len - len(sent))]
            features.append(feature)

        features = torch.LongTensor(features)

        return features, doc_lens

## Custom Dataset

In [13]:
class Dataset(data.Dataset):
    def __init__(self, examples):
        super(Dataset,self).__init__()
        # data: {'sents':xxxx,'labels':'xxxx', 'summaries':[1,0]}
        self.examples = examples
        self.training = False
    def train(self):
        self.training = True
        return self
    def test(self):
        self.training = False
        return self
    def shuffle(self,words):
        np.random.shuffle(words)
        return ' '.join(words)
    def dropout(self,words,p=0.3):
        l = len(words)
        drop_index = np.random.choice(l,int(l*p))
        keep_words = [words[i] for i in range(l) if i not in drop_index]
        return ' '.join(keep_words)
    def __getitem__(self, idx):
        ex = self.examples[idx]
        return ex

    def __len__(self):
        return len(self.examples)


# Main

## Training

In [14]:

def eval(net, vocab, data_iter, criterion):
    """
    Đánh giá mô hình trên tập dữ liệu.

    Args:
        net: Mô hình mạng nơ-ron.
        vocab: Từ điển.
        data_iter: Dữ liệu đánh giá.
        criterion: Hàm mất mát.

    Returns:
        Mất mát trung bình.
    """
    net.eval()  # Chuyển sang chế độ đánh giá
    total_loss = 0
    batch_num = 0

    for batch in data_iter:
        features, targets, _, doc_lens = vocab.make_features(batch) # Tạo đặc trưng
        features, targets = Variable(features), Variable(targets.float()) # Chuyển đổi sang biến
        if use_gpu: # Chuyển sang GPU
            features = features.cuda()
            targets = targets.cuda()
        probs = net(features, doc_lens) # Dự đoán
        loss = criterion(probs, targets) # Tính toán mất mát
        total_loss += loss.item()
        batch_num += 1

    loss = total_loss / batch_num # Tính mất mát trung bình
    net.train() # Chuyển lại chế độ huấn luyện
    return loss

In [15]:
# print(embedding_dir)  # In đường dẫn embedding
# print(kernel_sizes)  # In kích thước kernel

logging.info('Loading vocab,train and val dataset.Wait a second,please') # Thông báo tải dữ liệu

embed = torch.Tensor(np.load(embedding_dir)['embedding']) # Tải embedding
with open(word2id_dir) as f: # Tải word2id
    word2id = json.load(f)
vocab = Vocab(embed, word2id) # Tạo từ điển

with open(train_dir) as f: # Tải dữ liệu train
    train_examples = [json.loads(line) for line in f]
train_dataset = Dataset(train_examples[:10000]) # Tạo dataset train (10 examples)

with open(val_dir) as f: # Tải dữ liệu validation
    val_examples = [json.loads(line) for line in f]
val_dataset = Dataset(val_examples[0:1000]) # Tạo dataset validation (10 examples)

# update args
embed_num = embed.size(0) # Lấy số lượng embedding
embed_dim = embed.size(1) # Lấy kích thước embedding
# print(kernel_sizes)
kernel_sizes = [int(ks) for ks in kernel_sizes] # Chuyển đổi kích thước kernel thành int

# build model
# if you want to build a model with RNN-RNN achitecture
net = RNN_RNN(device, embed_num, embed_dim, hidden_size, seg_num, pos_num, pos_dim) # Tạo mô hình RNN-RNN
# if you want to build a model with CNN-RNN achitecture
# net = CNN_RNN(device, kernel_sizes, embed_dim, kernel_num, embed_num, hidden_size, seg_num, pos_num, pos_dim) # Tạo mô hình CNN-RNN (đang comment)

if use_gpu: # Chuyển mô hình sang GPU nếu cần
    net.cuda()

# load dataset
train_iter = DataLoader(dataset=train_dataset, # Tạo DataLoader cho train
                        batch_size=batch_size,
                        shuffle=True)
val_iter = DataLoader(dataset=val_dataset, # Tạo DataLoader cho validation
                        batch_size=batch_size,
                        shuffle=False)

# loss function
criterion = nn.BCELoss() # Hàm mất mát Binary Cross Entropy

# model info
# print(net) # In thông tin mô hình
params = sum(p.numel() for p in list(net.parameters())) / 1e6 # Tính số lượng tham số
# print('#Params: %.1fM' % (params)) # In số lượng tham số

min_loss = float('inf') # Khởi tạo mất mát tối thiểu
optimizer = torch.optim.Adam(net.parameters(),lr=lr) # Khởi tạo optimizer Adam
net.train() # Chuyển mô hình sang chế độ train

t1 = time() # Bắt đầu đo thời gian
for epoch in range(1, epochs + 1):  # Loop over epochs
    net.train()  # Set the model to training mode
    total_train_loss = 0  # Initialize variable to accumulate training loss
    for i, batch in enumerate(train_iter):  # Loop over batches in the training data
        features, targets, _, doc_lens = vocab.make_features(batch)  # Create features
        features, targets = Variable(features), Variable(targets.float())  # Convert to variables
        if use_gpu:  # Move to GPU if needed
            features = features.cuda()
            targets = targets.cuda()
        
        probs = net(features, doc_lens)  # Get predictions
        loss = criterion(probs, targets)  # Compute the loss
        optimizer.zero_grad()  # Reset gradients
        loss.backward()  # Backpropagation
        clip_grad_norm(net.parameters(), max_norm)  # Clip gradients
        optimizer.step()  # Update model parameters
        
        total_train_loss += loss.item()  # Accumulate the training loss
        
        if debug:  # Print batch information if debugging
            print('Batch ID:%d Loss:%f' % (i, loss.item()))
            continue
    
    # Compute average training loss for the epoch
    avg_train_loss = total_train_loss / len(train_iter)

    # Evaluate the model on the validation set
    net.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation during validation
        val_loss = eval(net, vocab, val_iter, criterion)  # Evaluate model on validation data
        if val_loss < min_loss: # Cập nhật mất mát tối thiểu
                min_loss = val_loss
                best_path = net.save() # Lưu mô hình tốt nhất
    
    # Print training and validation loss
    print('Epoch%2d Train_Loss: %f' % (epoch, avg_train_loss, ))
t2 = time() # Kết thúc đo thời gian
logging.info('Total Cost:%f h'%((t2-t1)/3600)) # In thời gian huấn luyện

<ipython-input-15-b317a4101892>:69: FutureWarning: `torch.nn.utils.clip_grad_norm` is now deprecated in favor of `torch.nn.utils.clip_grad_norm_`.
  clip_grad_norm(net.parameters(), max_norm)  # Clip gradients


Epoch 1 Train_Loss: 0.513053
Epoch 2 Train_Loss: 0.424046
Epoch 3 Train_Loss: 0.366028
Epoch 4 Train_Loss: 0.279440
Epoch 5 Train_Loss: 0.178280
Epoch 6 Train_Loss: 0.099140


## Test

In [16]:
embed = torch.Tensor(np.load(embedding_dir)['embedding']) # Tải embedding
with open(word2id_dir) as f: # Tải word2id
    word2id = json.load(f)
vocab = Vocab(embed, word2id) # Tạo từ điển

with open(test_dir) as f: # Tải dữ liệu test
    examples = [json.loads(line) for line in f]
test_dataset = Dataset(examples[0:10]) # Tạo dataset test 

test_iter = DataLoader(dataset=test_dataset, # Tạo DataLoader cho test
                        batch_size=batch_size,
                        shuffle=False)

if use_gpu: # Tải checkpoint mô hình (GPU)
    checkpoint = torch.load(load_dir)
else: # Tải checkpoint mô hình (CPU)
    checkpoint = torch.load(load_dir, map_location=lambda storage, loc: storage)

# checkpoint['args']['device'] saves the device used as train time
# if at test time, we are using a CPU, we must override device to None

net = RNN_RNN(device, embed_num, embed_dim, hidden_size, seg_num, pos_num, pos_dim) # Tạo mô hình RNN-RNN
# if you want to build a model with CNN-RNN achitecture
# net = CNN_RNN(device, kernel_sizes, embed_dim, kernel_num, embed_num, hidden_size, seg_num, pos_num, pos_dim) # Tạo mô hình CNN-RNN (đang comment)

net.load_state_dict(checkpoint['model']) # Tải trạng thái mô hình từ checkpoint
if use_gpu: # Chuyển mô hình sang GPU nếu cần
    net.cuda()
net.eval() # Chuyển mô hình sang chế độ đánh giá

doc_num = len(test_dataset) # Lấy số lượng tài liệu test
time_cost = 0 # Khởi tạo thời gian đánh giá
file_id = 1 # Khởi tạo id file kết quả

for batch in tqdm(test_iter): # Lặp qua các batch test
    features,_,summaries,doc_lens = vocab.make_features(batch) # Tạo đặc trưng
    t1 = time() # Bắt đầu đo thời gian dự đoán
    if use_gpu: # Dự đoán (GPU)
        probs = net(Variable(features).cuda(), doc_lens)
    else: # Dự đoán (CPU)
        probs = net(Variable(features), doc_lens)
    t2 = time() # Kết thúc đo thời gian dự đoán
    time_cost += t2 - t1 # Tính tổng thời gian dự đoán
    start = 0 # Khởi tạo vị trí bắt đầu
    for doc_id,doc_len in enumerate(doc_lens): # Lặp qua các tài liệu trong batch
        stop = start + doc_len # Tính vị trí kết thúc
        prob = probs[start:stop] # Lấy xác suất dự đoán cho tài liệu hiện tại
        topk = min(topk,doc_len) # Lấy top k câu (tối đa là độ dài tài liệu)
        topk_indices = prob.topk(topk)[1].cpu().data.numpy() # Lấy indices của top k câu
        topk_indices.sort() # Sắp xếp indices
        doc = batch['doc'][doc_id].split('\n')[:doc_len] # Lấy nội dung tài liệu
        # print(doc)
        # print("*" * 20)
        hyp = [doc[index] for index in topk_indices] # Tạo tóm tắt dự đoán
        # print(hyp)
        # print("*" * 20)
        ref = summaries[doc_id] # Lấy tóm tắt tham chiếu
        # print(ref)
        # print("-" * 80)

        os.makedirs(ref_dir, exist_ok=True) # Tạo thư mục tham chiếu nếu chưa tồn tại
        os.makedirs(hyp_dir, exist_ok=True) # Tạo thư mục dự đoán nếu chưa tồn tại
        with open(os.path.join(ref_dir,str(file_id)+'.txt'), 'w') as f: # Lưu tóm tắt tham chiếu
            f.write(ref)
        with open(os.path.join(hyp_dir,str(file_id)+'.txt'), 'w') as f: # Lưu tóm tắt dự đoán
            f.write('\n'.join(hyp))
        start = stop # Cập nhật vị trí bắt đầu
        file_id = file_id + 1 # Tăng id file
print('Speed: %.2f docs / s' % (doc_num / time_cost)) # In tốc độ đánh giá

<ipython-input-16-1cb9d56183a6>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_dir)
100%|██████████| 1/1 [00:00<00:00,  4.73it/s]

Speed: 65.27 docs / s


In [25]:
import glob
with open(test_dir) as f: # Tải dữ liệu train
    test_examples = [json.loads(line) for line in f]
    
def read_summaries(hyp_dir, ref_dir, file_id):
    """
    Đọc lại văn bản gốc, tóm tắt dự đoán và tóm tắt tham chiếu từ các tệp đã lưu.
    
    Args:
        hyp_dir (str): Thư mục chứa tóm tắt dự đoán.
        ref_dir (str): Thư mục chứa tóm tắt tham chiếu.
        file_id (int): ID của tệp cần đọc.
    
    Returns:
        tuple: (tóm tắt dự đoán, tóm tắt tham chiếu)
    """
    hyp_file = os.path.join(hyp_dir, f"{file_id}.txt")
    ref_file = os.path.join(ref_dir, f"{file_id}.txt")
    
    with open(hyp_file, 'r') as f:
        hyp_summary = f.read()
    
    with open(ref_file, 'r') as f:
        ref_summary = f.read()
    
    return hyp_summary, ref_summary

# Ví dụ đọc lại một tóm tắt
file_id_to_read = 1  # Thay đổi giá trị này để đọc file khác
hyp_text, ref_text = read_summaries(hyp_dir, ref_dir, file_id_to_read)
print("Văn bản gốc: ")
print(test_examples[file_id_to_read - 1])
print ("=" * 20)
print("Tóm tắt dự đoán:")
print(hyp_text)
print ("=" * 20)
print("\nTóm tắt tham chiếu:")
print(ref_text)

Văn bản gốc: 
{'doc': "things have n't been going well for manchester united flop anderson since he left old trafford to return to brazil , but at least on wednesday his own errors were outshone by the stupidity of a team - mate\nthe brazil midfielder was sent off for internacional after a first - half off - the - ball shove on ypiranga erechim striker neto that also saw his opponent sent off for retaliating with an elbow\nbut fortunately for anderson , his colleague fabricio stole the limelight by swearing at his own fans and becoming the third player to be shown the red card , reducing internacional to nine men in the 1 - 1 draw in porto alegre\nformer manchester united midfielder anderson shoved neto off the ball to earn a red card neto also saw red after he retaliated by by appearing to aim an elbow at anderson in porto alegre the two players listen to the referee as he reaches for his card from his back pocket on wednesday night anderson shows his shock to be shown the red card fo

In [20]:

test_examples[0]

{'doc': "things have n't been going well for manchester united flop anderson since he left old trafford to return to brazil , but at least on wednesday his own errors were outshone by the stupidity of a team - mate\nthe brazil midfielder was sent off for internacional after a first - half off - the - ball shove on ypiranga erechim striker neto that also saw his opponent sent off for retaliating with an elbow\nbut fortunately for anderson , his colleague fabricio stole the limelight by swearing at his own fans and becoming the third player to be shown the red card , reducing internacional to nine men in the 1 - 1 draw in porto alegre\nformer manchester united midfielder anderson shoved neto off the ball to earn a red card neto also saw red after he retaliated by by appearing to aim an elbow at anderson in porto alegre the two players listen to the referee as he reaches for his card from his back pocket on wednesday night anderson shows his shock to be shown the red card for what he beli

# Evaluation

In [17]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b74ec7bb882b558a9c10f016b0a0a43c160419ede80d1d3c93d9f780f3cbbf4b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [18]:
from rouge_score import rouge_scorer

# Tạo RougeScorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Danh sách lưu tổng điểm
total_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

# Duyệt qua tất cả các tệp trong thư mục tham chiếu
for filename in os.listdir(ref_dir):
    ref_path = os.path.join(ref_dir, filename)
    hyp_path = os.path.join(hyp_dir, filename)

    # Kiểm tra nếu tệp tồn tại trong cả hai thư mục
    if os.path.exists(hyp_path):
        with open(ref_path, 'r', encoding='utf-8') as f:
            reference = f.read().strip()
        with open(hyp_path, 'r', encoding='utf-8') as f:
            hypothesis = f.read().strip()

        # Tính điểm ROUGE cho từng cặp tệp
        scores = scorer.score(reference, hypothesis)

        # Lưu điểm vào danh sách
        for key in total_scores:
            total_scores[key].append(scores[key].fmeasure)  # Lưu giá trị F1-score

# Tính trung bình điểm ROUGE
avg_scores = {key: sum(values) / len(values) for key, values in total_scores.items()}

# In kết quả
print("ROUGE Average Scores:")
print(f"ROUGE-1: {avg_scores['rouge1']:.4f}")
print(f"ROUGE-2: {avg_scores['rouge2']:.4f}")
print(f"ROUGE-L: {avg_scores['rougeL']:.4f}")

ROUGE Average Scores:
ROUGE-1: 0.3625
ROUGE-2: 0.1889
ROUGE-L: 0.2416
